#🎮Trabajo Final -  Consultas Inteligentes sobre Jugadores del FIFA 23
Este trabajo final integra técnicas de procesamiento de lenguaje natural con modelos de embeddings y un LLM para permitir consultas en lenguaje natural sobre datos de jugadores del **FIFA 23**.

🧾 Los datos fueron obtenidos de un archivo CSV con estadísticas detalladas de jugadores como:
- Nombre del equipo
- Posiciones
- Liga
- Dorsal
- Estadísticas del jugador (valoracion, potencial, valor, velocidad)

⚙️ La app utiliza:
- **HuggingFace Embeddings** para vectorizar los datos (GPU si está disponible)
- **Gemini (Google)** como modelo de lenguaje para responder consultas ( Modelo: gemini-2.0-flash)
- **LlamaIndex** para realizar búsquedas semánticas sobre los datos
- **Streamlit** para la interfaz interactiva

##Instalacion de dependencias

In [1]:
!pip install llama-index llama-index-embeddings-huggingface llama-index-llms-langchain langchain-community langchain-google-genai google-generativeai pandas torch --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


##Lectura del dataset

In [2]:
import pandas as pd
# from google.colab import drive
# drive.mount('/content/drive')
# ruta_csv = "/content/drive/MyDrive/Trabajo Final Inteligentes - Juan/equipos_filtrados.csv"

df = pd.read_csv("jugadores_filtrados.csv")
df.head()

,id_jugador,nombre_corto,nombre_largo,posiciones_jugador,valoracion_general,potencial,valor_eur,salario_eur,edad,fecha_nacimiento,...,contencion_izq,contencion,contencion_der,lateral_der_ofensivo,lateral_izq,central_izq,central,central_der,lateral_der,arquero
0,158023,L. Messi,Lionel Andrés Messi Cuccittini,RW,91,91,54000000.0,195000.0,35,1987-06-24,...,63+3,63+3,63+3,64+3,59+3,50+3,50+3,50+3,59+3,19+3
1,165153,K. Benzema,Karim Benzema,"CF, ST",91,91,64000000.0,450000.0,34,1987-12-19,...,64+3,64+3,64+3,64+3,60+3,55+3,55+3,55+3,60+3,18+3
2,188545,R. Lewandowski,Robert Lewandowski,ST,91,91,84000000.0,420000.0,33,1988-08-21,...,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,192985,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,107500000.0,350000.0,31,1991-06-28,...,79+3,79+3,79+3,78+3,74+3,68+3,68+3,68+3,74+3,21+3
4,231747,K. Mbappé,Kylian Mbappé Lottin,"ST, LW",91,95,190500000.0,230000.0,23,1998-12-20,...,63+3,63+3,63+3,67+3,63+3,54+3,54+3,54+3,63+3,18+3


##Despliegue

In [3]:
!pip install streamlit pyngrok --quiet


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Creando .py para aplicacion Streamlit

In [1]:
%%writefile jugadores.py
import os
import pandas as pd
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from langchain_google_genai import ChatGoogleGenerativeAI
import torch
import streamlit as st
from llama_index.core import load_index_from_storage, VectorStoreIndex, Document, Settings
from llama_index.core.storage import StorageContext

os.environ["GOOGLE_API_KEY"] = st.secrets["GOOGLE_API_KEY"]
# os.environ["GOOGLE_API_KEY"] = "" #PONER API KEY DE GOOGLE

# Cargar los jugadores
df = pd.read_csv("jugadores_filtrados.csv")
#df = pd.read_csv("jugadores_RealMadrid.csv")

# Convertir cada fila a un documento de texto para indexar
documents = []
for _, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    documents.append(Document(text=text))

# Usar embeddings con GPU si hay disponible
device = "cuda" if torch.cuda.is_available() else "cpu"
embed_model = HuggingFaceEmbedding(
    model_name="sentence-transformers/all-mpnet-base-v2",#all-MiniLM-L12-v2/all-MiniLM-L6-v2/all-mpnet-base-v2
    device=device
)

# Cliente Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)

# Crear contexto de servicio
Settings.llm = llm
Settings.embed_model = embed_model

# Creando indice
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(similarity_top_k=3)

# # Guardar el indice
# from google.colab import drive
# drive.mount('/content/drive')
# persist_dir = "/content/drive/MyDrive/indice_jugadores"
# index.storage_context.persist(persist_dir=persist_dir)

#Cargar carpeta con indices para que tarde menos
# @st.cache_resource
# def cargar_query_engine():
#     # Cargar el storage context desde el directorio
#     storage_context = StorageContext.from_defaults(persist_dir="indice_jugadores")
#     # Cargar el indice desde el contexto
#     index = load_index_from_storage(storage_context)
#     # Crear el query engine
#     return index.as_query_engine(similarity_top_k=3)

# query_engine = cargar_query_engine()

# Interfaz de usuario
st.set_page_config(
    page_title="Consultas FIFA 23 - Jugadores",
    page_icon="⚽",
    layout="centered"
)

st.title("Consultas FIFA 23 - Jugadorees")
st.markdown("### 🧠 Ejemplos de preguntas que podes hacer:")

ejemplos = [
    "¿Cual es la valoracion de Lionel Messi?",
    "¿Cuanto vale en euros Kevin De Bruyne?",
    "¿Cuantos años tiene Luka Modric?",
    "¿En que posicion juega Dibu Martinez?",
    "¿Cual es la velocidad de Kylian Mbappe?",
    "¿Cual es el salario de Karim Benzema?",
    "¿Cual es la resistencia y fuerza de Neymar Jr?",
]

# Mostrar los ejemplos con botones
for i, ejemplo in enumerate(ejemplos):
    col1, col2 = st.columns([0.8, 0.2])
    with col1:
        st.markdown(f"- {ejemplo}")
    with col2:
        if st.button("Consultar", key=f"btn_{i}"):
            st.session_state["consulta"] = ejemplo

# Campo de entrada manual
consulta = st.text_input("Ingresa tu consulta:", value=st.session_state.get("consulta", ""))

# Ejecutar si hay una consulta
if consulta:
    with st.spinner("⏳ Procesando tu consulta..."):
        consulta_modificada = consulta.strip() + ". Responde en español."
        respuesta_llm = llm.invoke(consulta_modificada)
        respuesta_rag = query_engine.query(consulta_modificada)

    st.success("✅ Consulta completada")

    st.markdown("### 💬 Respuesta sin RAG:")
    st.write(respuesta_llm.content)

    st.markdown("### 🔍 Respuesta con RAG:")
    st.write(str(respuesta_rag))
    print(consulta_modificada, str(respuesta_rag))

Overwriting jugadores.py


## 🌐 Aplicacion publica con NGrok

In [ ]:
from pyngrok import ngrok, conf

conf.get_default().auth_token = "" # PONER TOKEN NGROK
# Crear tunel
public_url = ngrok.connect(addr="8501", proto="http")
print("URL publica:\n", public_url)
# Ejecutar Streamlit
!streamlit run app.py &
# 3 minutos aprox en levantar (cargando indices base2)

URL publica:
 NgrokTunnel: "https://6eb9-34-126-142-57.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.126.142.57:8501

2025-06-28 12:44:09.119134: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751114649.143652    6020 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751114649.151041    6020 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-28 12:44:09.175261: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performan

  Stopping...
